In [2]:
import pandas as pd

# load data

In [5]:
btcusdt = pd.read_csv('BTCUSDT_1h.csv')

In [6]:
btcusdt

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,num_trades,taker_base_vol,taker_quote_vol,ignore,order_book
0,1704067200000,42283.58,42554.57,42261.02,42475.23,1271.68108,1704070799999,5.395725e+07,47134,682.57581,2.895742e+07,0,NaN
1,1704070800000,42475.23,42775.00,42431.65,42613.56,1196.37856,1704074399999,5.098489e+07,50396,712.32227,3.035565e+07,0,NaN
2,1704074400000,42613.57,42638.41,42500.00,42581.10,685.21980,1704077999999,2.916738e+07,29863,288.98864,1.230102e+07,0,NaN
3,1704078000000,42581.09,42586.64,42230.08,42330.49,794.80391,1704081599999,3.370905e+07,38620,356.37209,1.511300e+07,0,NaN
4,1704081600000,42330.50,42399.99,42209.46,42399.99,715.41760,1704085199999,3.027162e+07,36038,371.12012,1.570362e+07,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,1715918400000,65584.53,65630.70,65367.51,65451.81,567.84995,1715921999999,3.719552e+07,28204,260.14101,1.704003e+07,0,NaN
3293,1715922000000,65451.81,65700.00,65436.00,65700.00,472.55691,1715925599999,3.097711e+07,27319,209.35999,1.372558e+07,0,NaN
3294,1715925600000,65699.99,66148.00,65580.66,65955.56,1212.84837,1715929199999,7.984624e+07,54481,748.41313,4.927709e+07,0,NaN
3295,1715929200000,65955.56,66500.00,65955.56,66386.94,2713.23533,1715932799999,1.797758e+08,82406,1519.21752,1.006827e+08,0,NaN


# drop columns

In [7]:
btcusdt.drop(columns=['ignore', 'order_book'], inplace=True)

# Convert 'open_time' and 'close_time' columns to datetime format

In [8]:
btcusdt['open_time'] = pd.to_datetime(btcusdt['open_time'], unit='ms')
btcusdt['close_time'] = pd.to_datetime(btcusdt['close_time'], unit='ms')

# Time Aggregation: Resample data to daily intervals

In [9]:
btcusdt.set_index('open_time', inplace=True)
btcusdt_resampled = btcusdt.resample('D').agg({
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    'quote_asset_volume': 'sum',
    'num_trades': 'sum',
    'taker_base_vol': 'sum',
    'taker_quote_vol': 'sum',
}).reset_index()

# Feature Engineering: Calculate additional features. eg. price_diff & price_change

In [10]:
btcusdt_resampled['price_diff'] = btcusdt_resampled['high'] - btcusdt_resampled['low']
btcusdt_resampled['price_change'] = btcusdt_resampled['close'] - btcusdt_resampled['open']

In [19]:
btcusdt_resampled

,open_time,open,high,low,close,volume,quote_asset_volume,num_trades,taker_base_vol,taker_quote_vol,price_diff,price_change
0,2024-01-01,42283.58,44184.10,42180.77,44179.55,27174.29903,1.169996e+09,1114623,14331.73180,6.173521e+08,2003.33,1895.97
1,2024-01-02,44179.55,45879.63,44148.34,44946.91,65146.40661,2.944332e+09,2247532,33817.14447,1.527964e+09,1731.29,767.36
2,2024-01-03,44946.91,45500.00,40750.00,42845.23,81194.55173,3.507105e+09,2658041,39103.99162,1.687665e+09,4750.00,-2101.68
3,2024-01-04,42845.23,44729.58,42613.77,44151.10,48038.06334,2.095095e+09,1819944,23605.90059,1.030075e+09,2115.81,1305.87
4,2024-01-05,44151.10,44357.46,42450.00,44145.11,48075.25327,2.100954e+09,2064845,24015.06426,1.049655e+09,1907.46,-5.99
...,...,...,...,...,...,...,...,...,...,...,...,...
133,2024-05-13,61484.00,63450.00,60749.21,62940.08,32733.41839,2.041897e+09,1371433,16717.90863,1.043061e+09,2700.79,1456.08
134,2024-05-14,62940.09,63118.36,61142.77,61577.49,29088.72041,1.800172e+09,1127939,13815.81443,8.550881e+08,1975.59,-1362.60
135,2024-05-15,61577.49,66444.16,61319.47,66206.50,43559.74719,2.794260e+09,1729454,21797.84094,1.398782e+09,5124.69,4629.01
136,2024-05-16,66206.51,66752.01,64602.77,65235.21,31106.36710,2.045508e+09,1269793,15645.11263,1.028962e+09,2149.24,-971.30


# Save btcusdt_resampled to a CSV file


In [13]:
btcusdt_resampled.to_csv('btcusdt_resampled_data.csv', index=False)

In [20]:
# create a file containing columns for refined data 
with open('column_info.txt', 'w') as file:
    # Iterate over columns and their data types
    for column in btcusdt_resampled.columns:
        col_info = f"{column} {btcusdt_resampled[column].dtype},\n"
        file.write(col_info)
with open('column_info.txt', 'r') as file:
    print(file.read())

open_time datetime64[ns],
open float64,
high float64,
low float64,
close float64,
volume float64,
quote_asset_volume float64,
num_trades int64,
taker_base_vol float64,
taker_quote_vol float64,
price_diff float64,
price_change float64,



In [17]:
btcusdt_resampled.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume',
       'quote_asset_volume', 'num_trades', 'taker_base_vol', 'taker_quote_vol',
       'price_diff', 'price_change'],
      dtype='object')